<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LightGBM</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#Тест-лучшей-модели" data-toc-modified-id="Тест-лучшей-модели-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Тест лучшей модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
!pip install -q transformers catboost

In [2]:
import os
import gc
import re
import sys

import pandas as pd
import numpy as np
from tqdm import notebook

import torch
import transformers
from transformers import BertTokenizer, BertModel, BertConfig

import sklearn
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample

import catboost
from catboost import CatBoostClassifier

import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
gc.enable()

In [4]:
RANDOM_STATE = 42
CV = 4

In [5]:
pth1 = '/datasets/toxic_comments.csv'
pth2 = '/Users/aleksandrfilippov/practicum/practicum_env/1datasets/toxic_comments.csv'
pth3 = '/content/drive/MyDrive/toxic_comments.csv'

if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
elif os.path.exists(pth3):
    df = pd.read_csv(pth3)
else:
    print('Something is wrong')

In [6]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


На этом этапе можно удалить столбец **Unnamed:0**, так как это дублирующий индекс столбец и привести столбец **text** к нижнему регистру

In [8]:
df_prep = df.drop('Unnamed: 0', axis=1)
df_prep['text'] = df_prep['text'].astype(str).str.lower()

Проверим длину текста в символах

In [9]:
df_prep.text.map(len).describe()

count    159292.000000
mean        393.691472
std         590.112598
min           5.000000
25%          95.000000
50%         205.000000
75%         435.000000
max        5000.000000
Name: text, dtype: float64

Рассмотрим баланс классов и попробуем его уровнять

In [11]:
df_prep['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Явный дисбаланс классов, алгоритм будет следующий:

- Найти дубликаты и удалить при их наличии
- В случае если дисбаланс сохранится, провести балансировку downsample

In [12]:
df_prep.duplicated(keep='first').sum()

45

In [13]:
df_prep = df_prep.drop_duplicates(keep='first').reset_index(drop=True)
df_prep['toxic'].value_counts(normalize=True)

0    0.898453
1    0.101547
Name: toxic, dtype: float64

Токенизируем комментарии и создадим эмбеддинги

In [15]:
tokenizer = BertTokenizer.from_pretrained('unitary/toxic-bert')

In [16]:
max_seq_length = 512

encoded_data = tokenizer.batch_encode_plus(
    df_prep['text'].tolist(),
    add_special_tokens=True,
    max_length=max_seq_length,
    pad_to_max_length=True,
    truncation=True
)

padded = np.array(encoded_data['input_ids'])
attention_mask = np.array(encoded_data['attention_mask'])
attention_mask.shape

(159247, 512)

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [18]:
config = BertConfig.from_pretrained('unitary/toxic-bert')
model = BertModel.from_pretrained('unitary/toxic-bert', config=config).to(device)

In [19]:
#Нашел код, показывающий распределение памяти, удалим не нужные данные
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                        padded: 622.1 MiB
                attention_mask: 622.1 MiB
                       df_prep: 92.1 MiB
                            df: 81.1 MiB
                 BertTokenizer:  2.0 KiB
                     BertModel:  2.0 KiB
                            _6:  1.8 KiB
                          _i19:  1.2 KiB
            RandomizedSearchCV:  1.0 KiB
            LogisticRegression:  1.0 KiB


In [20]:
del df

In [21]:
#Заменил размер батча, чтобы делилось без остатка и следующий код закомментировал
batch_size = 31
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size+1)):
        batch = torch.tensor(padded[batch_size*i:batch_size*(i+1)])
        attention_mask_batch = torch.tensor(attention_mask[batch_size*i:batch_size*(i+1)])

        with torch.no_grad():
            batch_embeddings = model(batch.to(device), attention_mask=attention_mask_batch.to(device))

        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/5138 [00:00<?, ?it/s]

По какой то причине, остаточные батчи(2) не были созданы, поэтому сделаем это вручную

In [23]:
features = np.concatenate(embeddings)
df_embedded = pd.DataFrame(features)
df_embedded['toxic'] = df_prep['toxic'].values

df_train, df_test = train_test_split(df_embedded,
                                     train_size=0.7,
                                     stratify = df_embedded['toxic'],
                                     random_state=RANDOM_STATE
                                     )

In [24]:
del df_prep
del padded
del attention_mask
del embeddings

In [25]:
toxic = df_train[df_train['toxic'] == 1]
non_toxic  = df_train[df_train['toxic'] == 0]
non_toxic_downsample = resample(non_toxic,
                                replace=True,
                                n_samples=len(toxic),
                                random_state=RANDOM_STATE)
df_train = pd.concat([toxic, non_toxic_downsample])

df_train['toxic'].value_counts(normalize=True)

1    0.5
0    0.5
Name: toxic, dtype: float64

In [26]:
x_train = df_train.drop('toxic', axis=1)
y_train = df_train['toxic']
x_test = df_test.drop('toxic', axis=1)
y_test = df_test['toxic']

### Вывод
В ходе выполнения первой части были проделаны следующие шаги:
- Удален лишний столбец Unnamed: 0
- Созданы эмбеддинги для моделей
- Выборки были разделены на обучающую и тестовую
- Проведена балансировка классов для тренировочной выборки

## Обучение

In [27]:
def best_params(model, params):
    model_name = RandomizedSearchCV(estimator = model,
                                    param_distributions = params,
                                    cv = CV,
                                    scoring='f1'
                                   )

    model_name.fit(x_train, y_train)

    best_model = model_name.best_params_
    best_score = model_name.best_score_
    print('Лучшие параметры:', best_model)
    print('f1-score:', best_score)

    return model_name

### Logistic Regression

In [28]:
%%time
hyper_params_lr ={
    'warm_start' : [True, False],
    'C' : np.arange(0, 1, 0.1)
}
model_lr = LogisticRegression(random_state=RANDOM_STATE)

model_lr_best = best_params(model_lr, hyper_params_lr)

Лучшие параметры: {'warm_start': False, 'C': 0.2}
f1-score: 0.9835005785613959
CPU times: user 1min 37s, sys: 11.7 s, total: 1min 49s
Wall time: 1min 6s


### LightGBM

In [29]:
%%time
hyper_params_lgb = {
                    'learning_rate': np.arange(0.1, 1, 0.1),
                    'boosting_type': ['gbdt', 'dart']
}

model_lgb = lgb.LGBMClassifier(random_state = RANDOM_STATE,
                               task = 'train',
                               verbosity=-1
                              )

model_lgb_best = best_params(model_lgb, hyper_params_lgb)

Лучшие параметры: {'learning_rate': 0.5, 'boosting_type': 'gbdt'}
f1-score: 0.9841564025697422
CPU times: user 12min 48s, sys: 2.54 s, total: 12min 50s
Wall time: 13min 12s


### CatBoostClassifier

In [34]:
%%time
hyper_params_cb = {
                    'learning_rate': [0.1,0.5,1]
}

model_cb = CatBoostClassifier(random_state=RANDOM_STATE, iterations=500)

model_cb_best = best_params(model_cb, hyper_params_cb)

Выходные данные были обрезаны до нескольких последних строк (5000).
4:	learn: 0.1441572	total: 2.09s	remaining: 3m 26s
5:	learn: 0.1186925	total: 2.45s	remaining: 3m 21s
6:	learn: 0.1001225	total: 2.86s	remaining: 3m 21s
7:	learn: 0.0873176	total: 3.22s	remaining: 3m 17s
8:	learn: 0.0784373	total: 3.6s	remaining: 3m 16s
9:	learn: 0.0718057	total: 3.9s	remaining: 3m 10s
10:	learn: 0.0661903	total: 4.13s	remaining: 3m 3s
11:	learn: 0.0618746	total: 4.34s	remaining: 2m 56s
12:	learn: 0.0598337	total: 4.55s	remaining: 2m 50s
13:	learn: 0.0576630	total: 4.76s	remaining: 2m 45s
14:	learn: 0.0558878	total: 4.98s	remaining: 2m 41s
15:	learn: 0.0544019	total: 5.21s	remaining: 2m 37s
16:	learn: 0.0534186	total: 5.4s	remaining: 2m 33s
17:	learn: 0.0522561	total: 5.61s	remaining: 2m 30s
18:	learn: 0.0509111	total: 5.82s	remaining: 2m 27s
19:	learn: 0.0501329	total: 6.01s	remaining: 2m 24s
20:	learn: 0.0493481	total: 6.22s	remaining: 2m 21s
21:	learn: 0.0487360	total: 6.42s	remaining: 2m 19s
22:	le

### Тест лучшей модели

In [35]:
predictions = model_lgb_best.predict(x_test)
print(f1_score(y_test,predictions))
confusion_matrix(y_test,predictions)

0.9111704155547083


array([[42063,   861],
       [   71,  4780]])

Лучшая модель показала результат в ~91%, что удовлетворяет требованиям, а матрица ошибок показывает, что ложные значения не сильно отличаются друг от друга. Так же стоит отметить, что наименьшее значение в матрице ошибок это ложно отрицательные, что лучшим образом подходит под критерий "найти токсичный комментарий для дальнейшей модерации"

### Вывод
В ходе данной части были выполнены следующие шаги:
- Обучена модель LogisticRegression, которая показала значение метрики f1=0.9835005785613959 при этих гиперпараметрах {'warm_start': False, 'C': 0.2}
- Обучена модель LGBMClassifier, которая показала значение метрики f1=0.9841564025697422 при этих гиперпараметрах {'learning_rate': 0.5, 'boosting_type': 'gbdt'}
- Обучена модель CatBoostClassifier, которая показала значение метрики f1=0.9835788450684699 при этих гиперпараметрах {'learning_rate': 0.1}
- Модель, которая показала лучший результат метрики(LGBMClassifier) была протестирована и показала результат f1=0.9111704155547083 , что удовлетворяет требованию заказчика
- Так же была рассмотрена матрица ошибок, чтобы определить значимость ложно отрицательных срабатываний(как самая критически важная ошибка)

## Выводы

**Небольшой оффтоп:**
Проверка на адекватность модели не имеет смысла, так как классы были сбалансированы и любой из типов dummyclassifier будет показывать 50%

В ходе работы были выполнены следующие шаги:

- Данные загружены и обработаны
    - Удален лишний столбец Unnamed: 0
    - Удалены строки текст которых составлял более 512 символов(ограничения BERT)
    - Проведена балансировка классов
    - Созданы эмбеддинги для моделей
    - Выборки были разделены на обучающую и тестовую
- Модели были обучены и выбрана наилучшая по целевой метрике
- Модель была протестирована и превысила целевой порог в 75%

**Как можно улучшить работу?**

Из-за нехватки мощностей устройства и частых сбоях ядра были сделаны следующие упрощения работы:
- Произведен downsampling с целью баланса классов И снижения энергопотребления -> Попробовать upsampling или другие методы(например синтезированные данные по векторам)
- Была использована упрощенная модель BERT`a, которая намного снижает нагрузку на систему, но в то же время не сильно отстает в результативности -> Выгружать более мощную модель и конфиги/модели
- Размер батчей слишком низок, что добавляет больше шума в данные -> Увеличить число батчей
- Подбор гиперпараметров был скуден из-за долгого времени поиска лучших гиперпараметров -> Увеличить сетку гиперпараметров